In [4]:
import openai
import unicodedata

In [5]:
# GPT api key
openai.api_key = "sk-q0Zt33VJIXp4OlOxB286Ff17E2D0435eA2A7198fA5CfB819"
openai.api_base = "https://lonlie.plus7.plus/v1"

In [7]:
# FactorGPT

messages = [
    {
        'role': 'system',
        'content': '您是一位全球顶级量化对冲基金的资深高级量化研究员，极其擅长挖掘和构造量价因子，'
                   '您的日常任务是从量价数据中构建蕴含高额稳定alpha的量价选股因子'
    }, 
    {
        'role': 'user', 
        'content': """
        请使用股票开盘价S_DQ_OPEN、收盘价S_DQ_CLOSE、最高价S_DQ_HIGH、最低价S_DQ_LOW、成交量S_DQ_VOLUME这五个数据，构建一个量价选股因子, 并给出【计算公式】和【含义】。
        
        您仅可使用以下这些算子来构建因子，通过对它们的组合、复合，可以构建出单个复杂且具备稳定高额alpha的因子。
         
        【算子名称】Add
        【算子函数签名】Add(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame
        【算子含义】将两个DataFrame 逐元素相加并返回结果。
        
        【算子名称】Sub
        【算子函数签名】Sub(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame
        【算子含义】从第一个 DataFrame 中逐元素减去第二个 DataFrame 并返回结果。
        
        【算子名称】Mul
        【算子函数签名】Mul(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame
        【算子含义】将两个 DataFrame 逐元素相乘并返回结果。
        
        【算子名称】Div
        【算子函数签名】Div(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame
        【算子含义】将第一个 DataFrame 逐元素除以第二个 DataFrame 并返回结果。
        
        下面是一个因子的例子，请仿照它的格式给出你构造的因子。
        【因子名称】改进的一致预期EPS变化率因子
        【因子计算方法】Div(ts_return(CON_EPS, 60),ts_decay_linear(DEGREE, 60))
        【因子含义】一致预期EPS(CON_EPS)在过去60个交易日的变化率除以过去60个交易日分析师预测标准差(DEGREE)的线性衰减加权值。"""
    }
]

# 生成因子，包括因子名称、计算方法、含义
response = openai.ChatCompletion.create(
    model='gpt-4-1106-preview',
    temperature=1,
    messages=messages
)

factor_message = response.choices[0].message
factor_message = unicodedata.normalize('NFKC', factor_message['content'])

print(f"FactorGPT:\n {factor_message}")

FactorGPT:
 【因子名称】量价动量因子

【因子计算方法】Div(Add(S_DQ_CLOSE, S_DQ_OPEN), Mul(Add(Sub(S_DQ_HIGH, S_DQ_LOW), S_DQ_VOLUME), 2))

【因子含义】这个因子考虑了股票的价格和交易量动量。公式的分子是当日股票的开盘价与收盘价之和,表示了股票一天的平均成交价格指标。分母则是股票最高价与最低价之差加上成交量再除以 2。最高价与最低价之差反映了价格震荡的幅度(价格振幅),而成交量给出了市场参与的活跃度。将价格平均值除以价格振幅和成交量的组合,以此反映价格变化趋势与市场活跃度之间的关系。这种量价结合可以指示资金的动向和市场情绪,特别是交易量在放大时可能暗示趋势的持续性,因此,因子值可能在一定程度上反映了市场的动量趋势。


In [10]:
# CodeGPT

messages = [
    {
        'role': 'system',
        'content': """
            您是一位全球顶级对冲基金公司精通Python的资深高级程序员，极其擅长编写Python代码，
            请您按照用户的要求编写计算量价因子的代码，要求完全准确无误。请严格按照下面的格式进行输出：
            '''python
            ...(你的python代码)
            ''' 
            例如: 
            '''python
            print('hello world')
            ''' 
        """
    }, 
    {
        'role': 'user', 
        'content':  """
                    下面是一个因子的计算步骤，请写出一个函数，其输入是计算所需要的数据，输出是因子的计算结果，输入输出的格式都是pandas.DataFrame。
                    请仿照下面的函数格式编写因子函数代码：
                    '''python
                    def Add(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:
                        result = x + y 
                        return result
                    '''
                    对于计算步骤中的算子，请务必调用其同名函数得到代码，并将其写在新的函数之前。这一点对于得到正确结果很重要。
                    因子的计算步骤如下：
                    Div(Add(S_DQ_CLOSE, S_DQ_OPEN), Mul(Add(Sub(S_DQ_HIGH, S_DQ_LOW), S_DQ_VOLUME), 2))
                    """
    }
]

# 生成因子计算代码
response = openai.ChatCompletion.create(
    model='gpt-4-1106-preview',
    temperature=0,
    messages=messages
)

code_message = response.choices[0].message

print(f"CodeGPT:\n {code_message}")


CodeGPT:
 {
  "content": "'''python\nimport pandas as pd\n\n# Add function\ndef Add(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:\n    result = x + y\n    return result\n\n# Sub function\ndef Sub(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:\n    result = x - y\n    return result\n\n# Mul function\ndef Mul(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:\n    result = x * y\n    return result\n\n# Div function\ndef Div(x: pd.DataFrame, y: pd.DataFrame) -> pd.DataFrame:\n    result = x / y\n    return result\n\n# Factor calculation function\ndef calculate_factor(S_DQ_CLOSE: pd.DataFrame, S_DQ_OPEN: pd.DataFrame, S_DQ_HIGH: pd.DataFrame, S_DQ_LOW: pd.DataFrame, S_DQ_VOLUME: pd.DataFrame) -> pd.DataFrame:\n    part1 = Add(S_DQ_CLOSE, S_DQ_OPEN)\n    part2 = Add(Sub(S_DQ_HIGH, S_DQ_LOW), S_DQ_VOLUME)\n    denominator = Mul(part2, 2)\n    factor = Div(part1, denominator)\n    return factor\n'''\n",
  "role": "assistant"
}


In [12]:
# EvalGPT

messages = [
    {
        'role': 'system',
        'content': """
            您是一位全球顶级对冲基金的资深高级研究员，负责因子的效果评估工作.
            请您根据量价因子的名称、计算方法、含义、回测结果, 给出对该因子的评估结果和优化建议.
            您的优化原则是: 1. 尽量提高量价因子的rankIC指标的绝对值 2. 尽量降低量价因子的rankIC指标的标准差 3. 尽量提高量价因子的因子收益率均值
            请按照下面的格式进行回答:
            【量价因子评估结果】: ...
            【量价因子优化建议】: ...   
        """
    }, 
    {
        'role': 'user', 
        'content':  """
        
        【因子名称】量价动量因子

        【因子计算方法】Div(Add(S_DQ_CLOSE, S_DQ_OPEN), Mul(Add(Sub(S_DQ_HIGH, S_DQ_LOW), S_DQ_VOLUME), 2))
        
        【因子含义】这个因子考虑了股票的价格和交易量动量。公式的分子是当日股票的开盘价与收盘价之和,表示了股票一天的平均成交价格指标。分母则是股票最高价与最低价之差加上成交量再除以 2。最高价与最低价之差反映了价格震荡的幅度(价格振幅),而成交量给出了市场参与的活跃度。将价格平均值除以价格振幅和成交量的组合,以此反映价格变化趋势与市场活跃度之间的关系。这种量价结合可以指示资金的动向和市场情绪,特别是交易量在放大时可能暗示趋势的持续性,因此,因子值可能在一定程度上反映了市场的动量趋势。
        
        【量价因子回测结果】
        IC均值: 0.009193750485056764
        IC标准差: 0.06474911363168545
        rank_IC均值: 0.0064362200676027605
        rank_IC标准差: 0.08436876614796869
        IC_IR: 0.14199036819799393
        多头IC均值: inf
        多头rank_IC均值: 0.005061443967738542
        空头IC均值: -0.010904636494557902
        空头rank_IC均值: -0.01758473987807314
        |t|均值: 2.7327786372585026
        |t|>2占比: 0.5327754532775453
        t均值: 0.4609717666847915
        因子收益率均值: 0.0005229134867147706
        R方均值: 0.004271117390165242
                    """
    }
]

# 生成因子评估结果和优化建议
response = openai.ChatCompletion.create(
    model='gpt-4-1106-preview',
    temperature=0,
    messages=messages
)

eval_message = response.choices[0].message
eval_message = unicodedata.normalize('NFKC', eval_message['content'])

print(f"EvalGPT:\n {eval_message}")


EvalGPT:
 【量价因子评估结果】: 根据回测结果,量价动量因子的IC均值为0.0092,表明因子与未来收益之间存在正相关关系,但相关性较弱。IC标准差为0.0647,说明因子表现的一致性不高,波动较大。rank_IC均值为0.0064,同样显示出正相关,但相关性更弱。rank_IC标准差为0.0844,表明排名信息系数的波动性更大。IC_IR为0.142,这个信息比率表明因子的预测能力较低。多头IC均值为无穷大,可能是由于数据异常或计算错误导致,需要进一步检查数据。空头IC均值为-0.0109,表明空头部分因子预测能力较弱。|t|均值为2.73,表明统计显著性较好,但|t|>2的占比为53.28%,说明有一半左右的时间因子表现不具备统计显著性。因子收益率均值为0.00052,表明因子的平均收益率正向且较低。R方均值为0.0043,解释度较低,表明因子对股票收益率变化的解释能力有限。

【量价因子优化建议】: 
1. 考虑到IC均值和rank_IC均值较低,可以尝试结合其他因子,如基本面因子或技术指标,以增强预测能力。
2. 由于IC标准差和rank_IC标准差较高,建议对因子进行平滑处理,比如使用过去一段时间的移动平均值来减少波动。
3. 对于多头IC均值异常的问题,需要重新检查数据和计算过程,确保没有错误。
4. 考虑到空头IC均值为负,可以探索因子在不同市场环境下的表现,比如牛市和熊市,以及不同市值股票的表现,以便更好地理解因子的行为。
5. 由于|t|>2的占比仅为53.28%,可以尝试使用更严格的统计显著性水平,如|t|>2.5或|t|>3,以提高因子的稳定性。
6. 因子收益率均值较低,可以通过优化股票池,剔除那些因子表现不佳的股票,或者通过动态调整因子权重来提高因子收益率。
7. R方均值较低,建议进一步研究因子的构成,可能需要对因子进行分解或重构,以提高其对股票收益率变化的解释能力。
